In [1]:
# !python3 -m venv .venv
# !pip install --upgrade pip
# !pip install boto3
# !pip install amazon-braket-pennylane-plugin
# !pip install sklearn
# !pip install pandas

In [1]:
# !pip install amazon-braket-pennylane-plugin

In [12]:
# import pandas as pd
import pennylane as qml
import pandas as pd
from pennylane import numpy as np


In [3]:
from pennylane.optimize import NesterovMomentumOptimizer

In [4]:
wires = 2  # Number of qubits
# dev = qml.device("braket.local.qubit", wires=wires)
dev = qml.device('default.qubit', wires=wires)

In [5]:
def layer(W):

    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

In [6]:
def get_angles(x):

    beta0 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[3] ** 2) / np.sqrt(x[2] ** 2 + x[3] ** 2 + 1e-12))
    beta2 = 2 * np.arcsin(
        np.sqrt(x[2] ** 2 + x[3] ** 2)
        / np.sqrt(x[0] ** 2 + x[1] ** 2 + x[2] ** 2 + x[3] ** 2)
    )

    return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2])

In [7]:
def statepreparation(a):
    qml.RY(a[0], wires=0)

    qml.CNOT(wires=[0, 1])
    qml.RY(a[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[2], wires=1)

    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[3], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[4], wires=1)
    qml.PauliX(wires=0)

In [8]:
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.CNOT(wires=[0, 1])

In [9]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [20]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [10]:
@qml.qnode(dev)
def circuit(weights, angles):
    statepreparation(angles)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))


def variational_classifier(weights, bias, angles):
    return circuit(weights, angles) + bias


def cost(weights, bias, features, labels):
    predictions = [variational_classifier(weights, bias, f) for f in features]
    return square_loss(labels, predictions)

In [13]:
# from sklearn.datasets import load_iris
# iris = load_iris()

# l_data = 100
# X = iris['data'][:l_data,:]
# y = iris['target'][:l_data]
# y_names = iris['target_names']
# X_names = iris['feature_names']

# data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
#                      columns= iris['feature_names'] + ['target'])[:l_data]
# data1  

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,1.0
96,5.7,2.9,4.2,1.3,1.0
97,6.2,2.9,4.3,1.3,1.0
98,5.1,2.5,3.0,1.1,1.0


In [14]:
data = np.loadtxt("iris_classes1and2_scaled.txt")
data

tensor([[ 0.4 ,  0.75,  0.2 ,  0.05, -1.  ],
        [ 0.3 ,  0.5 ,  0.2 ,  0.05, -1.  ],
        [ 0.2 ,  0.6 ,  0.15,  0.05, -1.  ],
        [ 0.15,  0.55,  0.25,  0.05, -1.  ],
        [ 0.35,  0.8 ,  0.2 ,  0.05, -1.  ],
        [ 0.55,  0.95,  0.35,  0.15, -1.  ],
        [ 0.15,  0.7 ,  0.2 ,  0.1 , -1.  ],
        [ 0.35,  0.7 ,  0.25,  0.05, -1.  ],
        [ 0.05,  0.45,  0.2 ,  0.05, -1.  ],
        [ 0.3 ,  0.55,  0.25,  0.  , -1.  ],
        [ 0.55,  0.85,  0.25,  0.05, -1.  ],
        [ 0.25,  0.7 ,  0.3 ,  0.05, -1.  ],
        [ 0.25,  0.5 ,  0.2 ,  0.  , -1.  ],
        [ 0.  ,  0.5 ,  0.05,  0.  , -1.  ],
        [ 0.75,  1.  ,  0.1 ,  0.05, -1.  ],
        [ 0.7 ,  1.2 ,  0.25,  0.15, -1.  ],
        [ 0.55,  0.95,  0.15,  0.15, -1.  ],
        [ 0.4 ,  0.75,  0.2 ,  0.1 , -1.  ],
        [ 0.7 ,  0.9 ,  0.35,  0.1 , -1.  ],
        [ 0.4 ,  0.9 ,  0.25,  0.1 , -1.  ],
        [ 0.55,  0.7 ,  0.35,  0.05, -1.  ],
        [ 0.4 ,  0.85,  0.25,  0.15, -1.  ],
        [ 

In [15]:
X = data[:, 0:2]
print("First X sample (original)  :", X[0])

# pad the vectors to size 2^2 with constant values
padding = 0.3 * np.ones((len(X), 1))
X_pad = np.c_[np.c_[X, padding], np.zeros((len(X), 1))]
print("First X sample (padded)    :", X_pad[0])

# normalize each input
normalization = np.sqrt(np.sum(X_pad ** 2, -1))
X_norm = (X_pad.T / normalization).T
print("First X sample (normalized):", X_norm[0])

# angles for state preparation are new features
features = np.array([get_angles(x) for x in X_norm], requires_grad=False)
print("First features sample      :", features[0])    

Y = data[:, -1]

First X sample (original)  : [0.4  0.75]
First X sample (padded)    : [0.4  0.75 0.3  0.  ]
First X sample (normalized): [0.44376016 0.83205029 0.33282012 0.        ]
First features sample      : [ 0.67858523 -0.          0.         -1.080839    1.080839  ]


In [22]:
features

tensor([[ 0.67858523, -0.        ,  0.        , -1.080839  ,  1.080839  ],
        [ 0.9503537 , -0.        ,  0.        , -1.03037683,  1.03037683],
        [ 0.88582209, -0.        ,  0.        , -1.24904577,  1.24904577],
        [ 0.96882903, -0.        ,  0.        , -1.30454428,  1.30454428],
        [ 0.6618505 , -0.        ,  0.        , -1.15838589,  1.15838589],
        [ 0.53355616, -0.        ,  0.        , -1.04600056,  1.04600056],
        [ 0.79365425, -0.        ,  0.        , -1.35970299,  1.35970299],
        [ 0.73210014, -0.        ,  0.        , -1.10714872,  1.10714872],
        [ 1.17034879, -0.        ,  0.        , -1.46013911,  1.46013911],
        [ 0.89317327, -0.        ,  0.        , -1.07144961,  1.07144961],
        [ 0.57615241, -0.        ,  0.        , -0.9964915 ,  0.9964915 ],
        [ 0.76721826, -0.        ,  0.        , -1.22777239,  1.22777239],
        [ 0.9850817 , -0.        ,  0.        , -1.10714872,  1.10714872],
        [ 1.080839  , -0.

In [16]:
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]
feats_val = features[index[num_train:]]
Y_val = Y[index[num_train:]]

# We need these later for plotting
X_train = X[index[:num_train]]
X_val = X[index[num_train:]]

In [17]:
num_qubits = 2
num_layers = 6

In [18]:
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

In [23]:
opt = NesterovMomentumOptimizer(0.01)
batch_size = 5

# train the variational classifier
weights = weights_init
bias = bias_init
for it in range(3):
    print(it)

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    print("bef")
    weights, bias, _, _ = opt.step(cost, weights, bias, feats_train_batch, Y_train_batch)
    print("aft")
    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(weights, bias, f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(weights, bias, f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(weights, bias, features, Y), acc_train, acc_val)
    )


0
bef
aft
Iter:     1 | Cost: 1.4749318 | Acc train: 0.4933333 | Acc validation: 0.5600000 
1
bef
aft
Iter:     2 | Cost: 1.4052149 | Acc train: 0.4933333 | Acc validation: 0.5600000 
2
bef
aft
Iter:     3 | Cost: 1.3297024 | Acc train: 0.4933333 | Acc validation: 0.5600000 
3
bef
aft
Iter:     4 | Cost: 1.2579575 | Acc train: 0.4933333 | Acc validation: 0.5600000 
4
bef
aft
Iter:     5 | Cost: 1.1261299 | Acc train: 0.4533333 | Acc validation: 0.5600000 
